In [1]:
## Import all the necessary libraries 
## ----------------------------------

import random
import plotly.subplots as sp
import plotly.graph_objs as go         
import numpy as np                     
import vtk
import ipywidgets as widgets
from ipywidgets import interactive

In [ ]:
## Load Data
## ------------------------------------------
temp = vtk.vtkXMLImageDataReader()          # 'temp' is a object of vtkXMLImageDataReader()
temp.SetFileName('mixture.vti')             #  Using 'temp' we will read the 'mixture.vti' file from given location
temp.Update()                               #  Updating the 'temp' so that we always get the updated file
vti_data = temp.GetOutput()                 #  We are initializing the output of the .vti file into a variable 
                                            #     named 'vti_data'
## ------------------------------------------

In [ ]:
## Converting the VTI data as a NumPy array
## ----------------------------------------
np_data = np.array(vti_data.GetPointData().GetScalars())    # Using the GetPointData(), GetScalars() function we can get into 
                                                            #   the vtidata and then in the PointData we can fetch the scalar
                                                            #   values.

np_data = np_data.reshape(vti_data.GetDimensions())   
np_data = np_data.transpose(2,1,0)                  # Here I am transposing the data as after converting the vti_data into
                                                    #   numpy data the correct order of dimensions of the numpy data got 
                                                    #   changed and to ensure it's correct order I am using transpose. 

In [ ]:
## Creating a 3D grid of points to use for coordinates
## ---------------------------------------------------
X, Y, Z = np.mgrid[:np_data.shape[0], :np_data.shape[1], :np_data.shape[2]] 

# Here I have used np.mgrid() function to create coordinate arrays to use it in Isosurface trace in Plotly

In [ ]:
## Creating a Plotly Isosurface trace
## ----------------------------------
                                           
isosurface = go.Isosurface(                # Using 'go.Isosurface' I am creating a Plotly Isosurface trace 
    x = X.flatten(),                       # Here I am flattening the X, Y, Z coordinate data to reshape the coordinate 
    y = Y.flatten(),                       #   arrays into one-dimensional arrays to match the format required by the trace
    z = Z.flatten(),
    value = np_data.flatten(),             # Passing the flattened np_data so that it can be connected to the respective
    opacity = 0.65,                        #    coordinate values
    isomin = 0.0,                          # Opacity value is set as 0.65 
    isomax = 0.0,                          # I have set both isomin and isomax to 0.0 as we need isovalue as 0.0
    surface_count = 3,                     # Setting maximum surfaces to 3
    colorscale = 'oranges',                       # Setting the colorscale to 'Oranges'
    caps = dict(x_show = False, y_show = False),  # Removing the caps when visualizing the isosurface
    showscale = False
    )

In [ ]:
## Creating Histogram trace
## ------------------------

histogram = go.Histogram(           # Using go.Histogram I am creating a Plotly Histogram trace
    x = np_data.flatten(),          # I am giving my numpy data after flattening it into one-D array
    nbinsx = 50,                    # I kept the bin size as 50 as it is the optimal bin size.
    marker = dict(                  
        color = 'blue',             # Making the histogram blue in colour
        opacity = 1                 # Made opacity as 1 as it's only bar 
    )
)

In [ ]:
## Creating subplot for Isosurface and Histogram
## ---------------------------------------------

fig = sp.make_subplots(rows = 1, cols = 2, specs = [[{'type': 'scene'}, {'type': 'xy'}]]) 

# Made 1st figure as 'scene' as it is for Isosurface and made 2nd figure 'xy' type as it is for Histogram


fig.add_trace(isosurface, row = 1, col = 1)   # Adding the Isosurface trace to Figure
fig.add_trace(histogram, row = 1, col = 2)    # Adding the Histogram trace to Figure

fig.update_layout(         
    scene = dict(               # Updating the layout for Isosurface
        xaxis_title = 'X',
        yaxis_title = 'Y',
        zaxis_title = 'Z',
    ),                          # Updating the layout for Histogram
    xaxis = dict(title = 'Vortex Scalar Values'),
    yaxis = dict(title = 'Frequency'),
)

In [ ]:
## Creating a Slider 
## -----------------

isovalue_slider = widgets.FloatSlider(
    value = 0.0, 
    min = np.min(np_data),       # For the range the existing data I have set the min and max
    max = np.max(np_data),
    step = 0.05,                 # Step count is set to 0.05
    description = 'Isoval:',      
    continuous_update = True,    # Continuous Update has been made 'True'
    readout = True,              
    readout_format = '.2f'
    
)
isovalue_slider.layout.width = '45%'     # Slider width has been made to 45 %

button = widgets.Button(description = "Reset")    # Making a reset button using widgets

In [ ]:
## Defining Update Funstion and Colour Change Functions
## ----------------------------------------------------

# Total number of colours supported by Plotly
total_colours = {'aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
             'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
             'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
             'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
             'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
             'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd'}


def colour_choice (set_c):           # Function for returning a random colour from the colour pool we have
    return random.choice(list(set_c))


def histo_update(isovalue):         # Function for updating the histogram according to the slider
    arr1 = []
    if (isovalue == 0.0):          # If equal to 0.0 then whole data is passed
        arr1 = np_data.flatten()
    else:                           # Otherwise the required range is passed
        arr1 = np_data[np.logical_and(np_data >= (isovalue - 0.25) , np_data <= (isovalue + 0.25))].flatten()
    return arr1
    
    
def update_isosurface(isovalue):     # Update function whenever slider value changes
    fig.data[0].isomin = isovalue    # Setting the Isosurface as per the isovalue changes
    fig.data[0].isomax = isovalue
    fig.data[0].colorscale = colour_choice(total_colours)   # Calling colour_choice function to get a colour
    fig.data[1].x = histo_update(isovalue)
                                     # Updating the Histogram according to the isovalue


def changing_isoval(x):                 # Defining an function which will update both the isosurface and Histogram 
    update_isosurface(x)   #   and then returning the figure
    return fig.show()

s = interactive(changing_isoval,x = isovalue_slider)  # Just a normal interactive ipywidget to call the changing_isoval
                                         #  function whenever isoval changes in the slider

def on_button_click(b):    # Function for reset button 
    isovalue_slider.value = 0.0    # Default value whenever reset is made to 0.0


button.on_click(on_button_click)   # Calling .on_click so that whenever it presses the reset button it calls
                                   #   the on_button_click() function

In [ ]:
display(button,s)    # Displaying the button and isovalue_slider with isosurface and Histogram